In [1]:
import matplotlib.pylab as plt
import numpy as np
import pandas as pd
import time
import sys
import os

import seaborn as sns
import matplotlib.dates as mdates
from datetime import datetime, date

# preamble for jupyter notebook and django
import django
import platform

if platform.node() == "srv-mcc-apsis":
    sys.path.append("/home/muef/tmv/BasicBrowser/")
    #sys.path.append('/home/leey/tmv/BasicBrowser/')
    dtm_path = "/home/galm/software/dtm/dtm/main"
else:
    # local paths
    sys.path.append('/media/Data/MCC/tmv/BasicBrowser/')
    dtm_path = "/home/finn/dtm/dtm/main"

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

# import from appended path
import parliament.models as pm
import scoping.models as sm
from parliament.tasks import do_search, run_tm, combine_searches
import cities.models as cmodels
from django.contrib.auth.models import User
from tmv_app.models import * 
from utils.tm_mgmt import update_topic_scores
from django.db.models import Q, Count, Func, F, Sum, Avg, Value as V
from django.db.models.functions import TruncDate, TruncMonth, TruncYear

In [11]:
run_id = 2350
no_topwords = 20

dts = DocTopic.objects.filter(run_id=run_id)
topic_period_scores = dts.values('topic__title','ut__document__parlperiod__n').annotate(score = Sum('score'))
df = pd.DataFrame.from_dict(topic_period_scores)
df.columns = ['score', 'topic', 'parlperiod']
df_scores = df.pivot(index='parlperiod', columns='topic', values='score')
df_scores.loc['peak_pos'] = [df_scores[topic].idxmax() for topic in df_scores.columns]
df_scores

topic,"{arbeit, arbeitslos, sozial, arbeitnehm, arbeitsplatz}","{atomenergi, grun, okolog, bundesregier, topf}","{bergbau, bergleut, bundesregier, steinkohlenbergbau, bergbaus}","{berlin, ddr, ost, bundesland, west}","{bundeskanzl, deutsch, partei, deutschland, regier}","{darauf, darub, haus, bereich, punkt}","{deutsch, bahn, bundesbahn, strass, verkehrspolit}","{deutsch, deutschland, staat, bundesregier, russland}","{deutsch, volk, bundesregier, freiheit, deutschland}","{energi, erneuerbar, kohl, energiepolit, energietrag}",...,"{saar, saarland, steinkohl, deutsch, bundesregier}","{sozial, wohnung, rent, gesetz, miet}","{spd, prozent, koalition, euro, deutschland}","{steu, kommun, belast, burg, entlast}","{subvention, handwerk, euro, mittelstand, milliard}","{tun, wiss, wirklich, red, gar}","{umweltschutz, umweltpolit, umwelt, luft, bundesregier}","{unternehm, wirtschaft, deutsch, deutschland, markt}","{wachstum, wirtschaft, wirtschaftspolit, bundesregier, aufschwung}","{wettbewerb, strom, kost, kraftwerk, preis}"
parlperiod,,,,,,,,,,,,,,,,,,,,,
1,2736.402013,6.250000,1203.456920,922.281888,20954.647690,29570.854228,8859.196201,16352.256080,7245.305109,109.967582,...,600.308102,30800.527274,278.643347,10550.394510,358.034158,34871.676295,170.747714,19527.988686,2490.244918,133.800955
2,1238.110016,86.727267,1861.159903,313.188712,8006.510075,32409.030139,25613.350759,15616.427434,5741.216564,426.521515,...,584.810340,12936.559855,130.706389,4918.252851,139.018589,27966.162279,347.870875,24322.678377,7387.399337,275.789180
3,1657.140432,58.160275,4956.503596,186.005579,5617.006743,17488.523667,12501.701518,4383.795790,4474.428901,577.540684,...,410.870345,9105.550617,113.227022,4515.475153,280.763891,22525.190183,511.383638,17077.986502,8461.146291,312.753859
4,614.034279,25.903885,11738.744518,171.682336,4695.553568,18788.790099,7260.055793,3997.449004,6018.334885,2156.023047,...,100.658659,3813.203321,101.488017,2632.750128,116.232722,15703.164858,310.482257,12161.850947,2588.572351,237.944980
5,2414.151357,55.216104,23042.731360,237.413062,11233.461062,41199.330774,7281.799161,4812.980086,16661.180502,2443.334634,...,1194.056244,5877.915911,175.361127,4268.223532,416.471753,28573.695703,236.665488,27829.584234,20705.233167,316.197648
6,549.790551,5.652296,1748.231997,89.480913,3588.836124,7000.517929,1151.901138,398.378012,4239.000571,695.091795,...,223.333101,1348.342930,16.062606,1179.077130,28.598703,4633.578567,252.086147,3389.087215,7628.944748,57.640590
7,2167.031025,32.703711,3401.591728,431.435108,7715.538453,25420.252463,1407.219583,1762.637854,9355.909323,24413.735954,...,3041.886766,2216.348206,75.202064,3058.782732,98.030403,14538.858250,859.264599,11545.858097,7805.871018,439.961721
8,4931.778273,172.521089,2936.949468,435.831256,19367.597013,34080.125027,1195.021452,3732.359708,12774.707864,38339.623418,...,4161.546177,1572.831387,211.853988,5575.587174,534.486899,22886.779974,4527.459971,16766.369416,5858.097935,640.487334
9,6429.237588,81.325656,945.164826,748.754320,13081.243237,16739.629201,556.799025,2767.600754,16585.572033,11395.546762,...,2723.664580,1132.967124,238.562049,2614.180030,495.473508,18802.583964,1356.437454,10582.879805,3049.506817,367.423473


In [12]:
TopicTimePeriodScores.objects.filter(
        topic=topic
    ).order_by('period__n')

<QuerySet [<TopicTimePeriodScores: TopicTimePeriodScores object (93154)>, <TopicTimePeriodScores: TopicTimePeriodScores object (92914)>, <TopicTimePeriodScores: TopicTimePeriodScores object (92874)>, <TopicTimePeriodScores: TopicTimePeriodScores object (92954)>, <TopicTimePeriodScores: TopicTimePeriodScores object (92594)>, <TopicTimePeriodScores: TopicTimePeriodScores object (92634)>, <TopicTimePeriodScores: TopicTimePeriodScores object (92674)>, <TopicTimePeriodScores: TopicTimePeriodScores object (92714)>, <TopicTimePeriodScores: TopicTimePeriodScores object (92754)>, <TopicTimePeriodScores: TopicTimePeriodScores object (92794)>, <TopicTimePeriodScores: TopicTimePeriodScores object (92834)>, <TopicTimePeriodScores: TopicTimePeriodScores object (92994)>, <TopicTimePeriodScores: TopicTimePeriodScores object (93034)>, <TopicTimePeriodScores: TopicTimePeriodScores object (93074)>, <TopicTimePeriodScores: TopicTimePeriodScores object (93114)>, <TopicTimePeriodScores: TopicTimePeriodScore

In [13]:
# topic labeling: manual and automatic

labels = {}

df = pd.DataFrame(columns=['label', 'top words avg', 'top words peak', 'score pct', 'score'])

get_input = False

# set to 0 if current topic labels should be kept
no_label_topwords = 5

for topic in Topic.objects.filter(run_id = run_id).order_by('-score'):
    stat = RunStats.objects.get(run_id=run_id)

    print("Topic #{}".format(topic.id))
    
    if stat.method == 'BD':
        
        terms = Term.objects.filter(topicterm__topic=topic).values('title').annotate(
                    summed_score = Sum(F('topicterm__score'))).order_by('-summed_score')[:no_topwords]
        term_list = [t['title'] for t in terms]
        print(", ".join(term_list))
        
        """
        # todo: weight by document scores per period -> how to get this?
        weighted_terms = TopicTerm.objects.filter(topic=topic).values('term').annotate(
                        summed_score = Sum(F('score') * F('PY__topictimeperiodscores__score'))).order_by('-summed_score')[:20].select_related('term')        

        weighted_term_list = [t.term.title for t in weighted_terms]
        print("weighted by topic scores:")
        print(", ".join(weighted_term_list))
        """
        
        # top words from peak
        py = df_scores.loc['peak_pos', topic.title] - 1
        # note: PY starts with 0
        
        peak_terms = TopicTerm.objects.filter(topic=topic, PY=py
                                            ).order_by('-score')[:no_topwords].select_related('term')
        
        
        peak_term_list = [t.term.title for t in peak_terms]
        print("peak terms:")
        print(", ".join(peak_term_list))

    else:
        print("Top words: {}".format(topic.top_words))

    if get_input:
        label = input("topic label: ")
        if label:
            labels[topic.id] = label
            topic.title = label
            topic.save()
              
    elif no_label_topwords:
        topic.title = "{" + ", ".join(term_list[:no_label_topwords]) + "}"
        topic.save()
        print(topic.title)
    
    else:
        print("topic label: {}".format(topic.title))
    
    print("")
    
    df.loc[topic.id] = {"top words avg": ", ".join(term_list), "top words peak": ", ".join(peak_term_list),
                        "score": topic.score, "label": topic.title, "score pct": 0}

print(labels)
df['score pct'] = 100 * df['score'] / df['score'].sum()
df = df.sort_values(by=['score'], ascending=False)

Topic #184089
tun, wiss, wirklich, red, gar, darub, natur, richtig, eigent, namlich, vielleicht, weiss, uberhaupt, seh, stell, ding, sei, einfach, sagt, gemacht
peak terms:
ding, vielleicht, wirklich, gar, wiss, seh, weiss, sei, darub, tun, richtig, sagt, namlich, wort, wenig, genau, eben, kommt, uberhaupt, eigent
{tun, wiss, wirklich, red, gar}

Topic #184090
darauf, darub, haus, bereich, punkt, debatt, bundesregier, entscheid, deutlich, diskussion, darf, richtig, natur, zusammenhang, wichtig, ausdruck, stell, seit, bundestag, lass
peak terms:
haus, darauf, darf, darub, entscheid, lass, bundesregier, seit, meinung, punkt, vorschlag, debatt, zusammenhang, klar, stell, regier, ausfuhr, natur, ausdruck, vielleicht
{darauf, darub, haus, bereich, punkt}

Topic #184081
unternehm, wirtschaft, deutsch, deutschland, markt, entwickl, wettbewerb, marktwirtschaft, staat, stark, wettbewerbsfah, investition, wirtschaftspolit, staatlich, bereich, entscheid, bundesregier, hoh, insbesond, schwierig
pe

deutsch, deutschland, staat, bundesregier, russland, sowjetunion, militar, aussenpolit, bundesrepubl, krieg, sudafrika, regier, bundeswehr, seit, soldat, vereinigt, ost, sowjet, lag, nation
peak terms:
deutschland, deutsch, west, ost, bundesrepubl, sowjet, staat, krieg, militar, wiederverein, macht, sowjetunion, seit, westlich, letzt, verteid, recht, stark, kommunist, aussenpolit
{deutsch, deutschland, staat, bundesregier, russland}

Topic #184100
saar, saarland, steinkohl, deutsch, bundesregier, kohl, strukturwandel, kommission, verstromungsgesetz, arbeitsplatz, stahlindustri, heimisch, kohlepolit, nordrheinwestfal, zukunft, bundeswirtschaftsminist, elektrizitatswirtschaft, erhalt, kohlepfenn, end
peak terms:
kohl, bundesregier, deutsch, steinkohl, jahrhundertvertrag, kohlepfenn, kohlepolit, heimisch, saarland, arbeitsplatz, stahlindustri, bundeswirtschaftsminist, kons, nordrheinwestfal, energiepolit, verstrom, zukunft, kohlerund, erhalt, beteiligt
{saar, saarland, steinkohl, deutsch,

In [14]:
df

,label,top words avg,top words peak,score pct,score
184089,"{tun, wiss, wirklich, red, gar}","tun, wiss, wirklich, red, gar, darub, natur, r...","ding, vielleicht, wirklich, gar, wiss, seh, we...",10.672064,419821.116446
184090,"{darauf, darub, haus, bereich, punkt}","darauf, darub, haus, bereich, punkt, debatt, b...","haus, darauf, darf, darub, entscheid, lass, bu...",9.348242,367744.157874
184081,"{unternehm, wirtschaft, deutsch, deutschland, ...","unternehm, wirtschaft, deutsch, deutschland, m...","wirtschaft, entwickl, bereich, wirtschaftspoli...",5.492435,216063.202495
184105,"{bundeskanzl, deutsch, partei, deutschland, re...","bundeskanzl, deutsch, partei, deutschland, reg...","regier, bundeskanzl, volk, partei, sozialdemok...",4.037749,158838.262969
184102,"{deutsch, volk, bundesregier, freiheit, deutsc...","deutsch, volk, bundesregier, freiheit, deutsch...","deutsch, deutschland, gemeinsam, wirtschaft, f...",3.989937,156957.446933
184106,"{milliard, bund, million, hoh, rund}","milliard, bund, million, hoh, rund, haushalt, ...","million, milliard, bund, haushalt, hoh, bundes...",3.783282,148827.974002
184086,"{europa, union, gemeinschaft, deutsch, vertrag}","europa, union, gemeinschaft, deutsch, vertrag,...","deutsch, schumanplan, deutschland, vertrag, fr...",3.685528,144982.504003
184098,"{europa, gemeinsam, international, staat, geme...","europa, gemeinsam, international, staat, gemei...","deutschland, wichtig, international, gemeinsam...",3.627137,142685.511866
184108,"{energi, erneuerbar, kohl, energiepolit, energ...","energi, erneuerbar, kohl, energiepolit, energi...","energiepolit, bundesregier, energi, kohl, kern...",3.422472,134634.299691
184097,"{gesetz, wirtschaft, bundesregier, entwurf, bu...","gesetz, wirtschaft, bundesregier, entwurf, bun...","ewg, rechtzeit, gesetz, endgult, wirtschaft, b...",3.160606,124332.949316


In [18]:
fname = "./plots/topic_analysis_{}/0_topic_list+peak_topwords_run{}.xlsx".format(run_id, run_id)
writer = pd.ExcelWriter(fname, engine='xlsxwriter')

df.to_excel(writer)

In [16]:
pd.set_option('max_colwidth', 500)

fname = "./plots/topic_analysis_{}/0_topic_list.tex".format(run_id)

with open(fname,'w') as tf:
    tf.write(df.to_latex(index=False, columns=["label", "top words", "score pct"],
                         column_format='L{3cm}L{10.5cm}r', float_format="{:0.2f}".format,
                        longtable=True).replace('\n',
                                                '\n\\caption{List of topics with labels, top words and scores (in percent of total score).}
                                                '\n\label{tab:topic-list}\n\small\\\\\n', 1))

SyntaxError: EOL while scanning string literal (<ipython-input-16-f0b5b4e3004e>, line 9)

In [17]:
# modify single labels:

topic = Topic.objects.get(id=178480)
topic.title = 'nuclear phase out 1'
topic.save()